<a href="https://colab.research.google.com/github/hasanzeynal/Tensorflow-Developer-Certificate/blob/main/01_Neural_Network_Regression_in_TensorFlow/neural_network_regression_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Regression with Neural Network in TensorFlow

In [1]:
#import tensorflow
import tensorflow as tf
print(tf.__version__)

2.12.0


# Basic Neural Network Model for Regression Problem(Car Price Prediction)
Then, After building our basic NN model, we are gonna try:
* improving our model
* evaluating
* setting
* saving
* loading
* processing with feature scalling(normalising, fitting model on a scaled data)

In [54]:
#ignore all warnings
import warnings
warnings.filterwarnings('ignore')
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [70]:
#load test and train dataset
test_df = pd.read_csv('/content/drive/MyDrive/datasets/Car_price/test.csv')
train_df = pd.read_csv('/content/drive/MyDrive/datasets/Car_price/train.csv')

In [71]:
test_df.shape,train_df.shape

((8245, 18), (19237, 18))

In [72]:
test_df.head()

,ID,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Price
0,44020629,-,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0 Turbo,0 km,4,Manual,Front,02-Mar,Left wheel,Grey,10,NaN
1,45784798,-,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000 km,4,Tiptronic,Front,04-May,Left wheel,Grey,10,NaN
2,45800257,-,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000 km,4,Automatic,Front,04-May,Right-hand drive,Sky blue,8,NaN
3,45797981,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000 km,6,Automatic,4x4,04-May,Left wheel,Blue,12,NaN
4,45814303,-,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000 km,4,Manual,Front,04-May,Left wheel,Black,0,NaN


In [73]:
train_df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


## Data Preperation

In [74]:
#defining target variable
y = train_df['Price']

In [75]:
#drop the prices
test_df.drop('Price',axis=1,inplace=True)
train_df.drop('Price',axis=1,inplace=True)
#merge two df
X = test_df.append(train_df)
X.reset_index(inplace=True)
#drop id and index from X
X.drop(['ID','index'],axis=1,inplace=True)

In [76]:
X.head()

,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,-,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0 Turbo,0 km,4.0,Manual,Front,02-Mar,Left wheel,Grey,10
1,-,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000 km,4.0,Tiptronic,Front,04-May,Left wheel,Grey,10
2,-,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000 km,4.0,Automatic,Front,04-May,Right-hand drive,Sky blue,8
3,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000 km,6.0,Automatic,4x4,04-May,Left wheel,Blue,12
4,-,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000 km,4.0,Manual,Front,04-May,Left wheel,Black,0


In [77]:
#checking for duplicate and null values
X.duplicated().sum(),X.isnull().any().sum()

(6253, 0)

In [78]:
#create new dataframe to show number of unique values in the dataset
inform = pd.DataFrame()
inform['Type of Column'] = X.dtypes
inform['Number of Uniques'] = X.nunique()
inform

,Type of Column,Number of Uniques
Levy,object,635
Manufacturer,object,68
Model,object,1981
Prod. year,int64,59
Category,object,11
Leather interior,object,2
Fuel type,object,7
Engine volume,object,112
Mileage,object,9977
Cylinders,float64,16


In [79]:
X

,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,-,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0 Turbo,0 km,4.0,Manual,Front,02-Mar,Left wheel,Grey,10
1,-,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000 km,4.0,Tiptronic,Front,04-May,Left wheel,Grey,10
2,-,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000 km,4.0,Automatic,Front,04-May,Right-hand drive,Sky blue,8
3,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000 km,6.0,Automatic,4x4,04-May,Left wheel,Blue,12
4,-,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000 km,4.0,Manual,Front,04-May,Left wheel,Black,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27477,-,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0 Turbo,300000 km,4.0,Manual,Rear,02-Mar,Left wheel,Silver,5
27478,831,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600 km,4.0,Tiptronic,Front,04-May,Left wheel,Red,8
27479,836,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2,116365 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
27480,1288,CHEVROLET,Captiva,2007,Jeep,Yes,Diesel,2,51258 km,4.0,Automatic,Front,04-May,Left wheel,Black,4


In [80]:
#change the column names
X.columns = [i.lower() for i in X.columns]
#replace all space with -
X.columns = [i.replace(' ','_') for i in X.columns]
X.columns

Index(['levy', 'manufacturer', 'model', 'prod._year', 'category',
       'leather_interior', 'fuel_type', 'engine_volume', 'mileage',
       'cylinders', 'gear_box_type', 'drive_wheels', 'doors', 'wheel', 'color',
       'airbags'],
      dtype='object')

In [81]:
X.head()

,levy,manufacturer,model,prod._year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags
0,-,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0 Turbo,0 km,4.0,Manual,Front,02-Mar,Left wheel,Grey,10
1,-,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000 km,4.0,Tiptronic,Front,04-May,Left wheel,Grey,10
2,-,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000 km,4.0,Automatic,Front,04-May,Right-hand drive,Sky blue,8
3,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000 km,6.0,Automatic,4x4,04-May,Left wheel,Blue,12
4,-,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000 km,4.0,Manual,Front,04-May,Left wheel,Black,0


In [82]:
# delete Trubo from engine volume
X['engine_volume'] = [i.split(' ')[0] for i in X['engine_volume']]
# delete km from milage
X['mileage'] = [i.split(' ')[0] for i in X['mileage']]
# change Mar and May from doors
X['doors'] = X['doors'].replace({'02-Mar':2,'04-May':4})
# replace - with 0 in the levy column
X['levy'] = X['levy'].replace({'-':0})

In [83]:
X.head()

,levy,manufacturer,model,prod._year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags
0,0,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0,0,4.0,Manual,Front,2,Left wheel,Grey,10
1,0,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000,4.0,Tiptronic,Front,4,Left wheel,Grey,10
2,0,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000,4.0,Automatic,Front,4,Right-hand drive,Sky blue,8
3,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000,6.0,Automatic,4x4,4,Left wheel,Blue,12
4,0,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000,4.0,Manual,Front,4,Left wheel,Black,0


In [84]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27482 entries, 0 to 27481
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   levy              27482 non-null  object 
 1   manufacturer      27482 non-null  object 
 2   model             27482 non-null  object 
 3   prod._year        27482 non-null  int64  
 4   category          27482 non-null  object 
 5   leather_interior  27482 non-null  object 
 6   fuel_type         27482 non-null  object 
 7   engine_volume     27482 non-null  object 
 8   mileage           27482 non-null  object 
 9   cylinders         27482 non-null  float64
 10  gear_box_type     27482 non-null  object 
 11  drive_wheels      27482 non-null  object 
 12  doors             27482 non-null  object 
 13  wheel             27482 non-null  object 
 14  color             27482 non-null  object 
 15  airbags           27482 non-null  int64  
dtypes: float64(1), int64(2), object(13)
memo

In [85]:
# change dtype from the categorical to the numeric
X['levy'] = X['levy'].astype('int64')
X['engine_volume'] = X['engine_volume'].astype('float64')
X['mileage'] = X['mileage'].astype('int64')

In [86]:
# create numerical and categorical features
cat_features = [i for i in X.columns if X[i].dtype == 'object']
num_features = [i for i in X.columns if X[i].dtype != 'object']

In [87]:
#get dummies on categorical features
cat_df = X[cat_features]
dummied_cat = pd.get_dummies(cat_df,drop_first=True)
num_df = X[num_features]

In [88]:
X = pd.merge(num_df, dummied_cat, left_index=True, right_index=True)
X.head()

,levy,prod._year,engine_volume,mileage,cylinders,airbags,manufacturer_ALFA ROMEO,manufacturer_ASTON MARTIN,manufacturer_AUDI,manufacturer_BENTLEY,...,color_Green,color_Grey,color_Orange,color_Pink,color_Purple,color_Red,color_Silver,color_Sky blue,color_White,color_Yellow
0,0,2012,2.0,0,4.0,10,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,2012,2.4,26000,4.0,10,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,2005,1.5,168000,4.0,8,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,975,2012,3.2,143000,6.0,12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1993,1.6,200000,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
X.shape

(27482, 2093)

## Building Basic Model in TensorFlow

### Steps in Modelling in TensorFlow.
1. **Creating Model** - define imput, output, hidden layers of our Deep Learning Model.

2. **Compiling Model** - define the loss function(the function which tell us how wrong our model is), define optimizer(which tell us our model how improve) and evaluation metrics(we can use to interpret the performance of our model).

3. Fitting Model - letting the model try to find patterns between X and y(features and labels)

In [90]:
X.shape,y.shape

((27482, 2093), (19237,))

In [91]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.2,random_state = 42)

ValueError: ignored

In [ ]:
# 1.Create a model using Sequential API
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8,input_shape = (2093,)))
model.add(tf.keras.layers.Dense(1))

# 2.Compile the model,pass loss,optimizer and metrics
model.compile(loss= tf.keras.losses.BinaryCrossentropy(), #MAE stands for Mean Absolute Error
              optimizer= tf.keras.optimizers.SGD(),
              metrics = ['mae']) #SGD stands for Stochastic gradient Decsent

# 3.fit the model,for the first time X_train and y train is gonna be enough
model.fit(X,y,epochs = 15)